In [ ]:
import cv2
import pytesseract
import re

def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to preprocess the image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    
    return thresh

def extract_text_from_image(image):
    # Use Tesseract to do OCR on the image
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(image, config=custom_config)
    return text

def find_name_and_id(text):
    # Assuming the name and ID number follow a specific pattern
    # This will vary depending on the format of the ID card
    name_pattern = r"1,2.\s*(.*)"
    id_pattern = r"ID\s*:\s*(\d+)"
    
    name_match = re.search(name_pattern, text)
    id_match = re.search(id_pattern, text)
    
    name = name_match.group(1) if name_match else "Not found"
    id_number = id_match.group(1) if id_match else "Not found"
    
    return name, id_number

def main(image_path):
    # Preprocess the image
    processed_image = preprocess_image(image_path)
    
    # Extract text from the image
    text = extract_text_from_image(processed_image)
    print("Extracted Text:")
    print(text)
    
    # Find the name and ID
    name, id_number = find_name_and_id(text)
    print(f"Name: {name}")
    print(f"ID Number: {id_number}")

# Replace 'path_to_id_card_image.jpg' with the path to your ID card image
main('path_to_id_card_image.jpg')



In [3]:
import cv2
import pytesseract

def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
    return thresh

def find_text_regions(image):
    # Find contours
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours based on their area (larger first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    return contours

def extract_name(image, contours):
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = image[y:y+h, x:x+w]
        
        # Use Tesseract to do OCR on the ROI
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(roi, config=custom_config)
        
        # Check if the extracted text matches the expected name format
        if is_valid_name(text):
            return text.strip()
    
    return "Name not found"

def is_valid_name(text):
    # Assuming a valid name contains only alphabetic characters and spaces
    return bool(text) and all(char.isalpha() or char.isspace() for char in text)

def main(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    
    # Find text regions
    contours = find_text_regions(preprocessed_image)
    
    # Extract the name from the identified regions
    name = extract_name(preprocessed_image, contours)
    
    print(f"Extracted Name: {name}")

# Path to the uploaded image
image_path = 'licence1.jpg'
main(image_path)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [4]:
from data_store import register
import requests

def extract_file_id(drive_url):
    """
    Extract the file ID from a Google Drive URL.
    """
    return drive_url.split('/d/')[1].split('/')[0]

def construct_direct_download_url(file_id):
    """
    Construct a direct download URL from a Google Drive file ID.
    """
    return f'https://drive.google.com/uc?export=download&id={file_id}'

def download_image(download_url, output_path):
    """
    Download the image from the given URL and save it to the output path.
    """
    response = requests.get(download_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Image successfully downloaded: {output_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

def get_user_by_nic(register, nic):
    """
    Find the user in the register by NIC number.
    """
    for person in register['registered']:
        if person['NIC'] == nic:
            return person
    return None

def main():
    # Get user NIC number as input
    user_nic = input("Enter the NIC number: ")
    
    # Find the user by NIC
    user = get_user_by_nic(register, user_nic)
    
    if user:
        # Extract the Google Drive link
        id_image_link = user.get('id_image')
        if id_image_link:
            # Extract the file ID and construct the download link
            file_id = extract_file_id(id_image_link)
            download_url = construct_direct_download_url(file_id)
            
            # Download the image
            output_path = f"download/download_image.jpg"
            download_image(download_url, output_path)
        else:
            print("ID image link not found for this user.")
    else:
        print("User not found with the given NIC number.")

if __name__ == "__main__":
    main()

Image successfully downloaded: download/download_image.jpg


In [10]:
import os
import shutil
import requests
from data_store import register

def extract_file_id(drive_url):
    """
    Extract the file ID from a Google Drive URL.
    """
    return drive_url.split('/d/')[1].split('/')[0]

def construct_direct_download_url(file_id):
    """
    Construct a direct download URL from a Google Drive file ID.
    """
    return f'https://drive.google.com/uc?export=download&id={file_id}'

def download_image(download_url, output_path):
    """
    Download the image from the given URL and save it to the output path.
    """
    response = requests.get(download_url)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Image successfully downloaded: {output_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

def get_user_by_nic(register, nic):
    """
    Find the user in the register by NIC number.
    """
    for person in register['registered']:
        if person['NIC'] == nic:
            return person
    return None

def clear_download_folder(folder_path):
    """
    Remove all files in the specified folder.
    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

def final_output(user_nic):
    
    # Find the user by NIC
    user = get_user_by_nic(register, user_nic)
    
    if user:
        # Extract the Google Drive link
        id_image_link = user.get('id_image')
        if id_image_link:
            # Extract the file ID and construct the download link
            file_id = extract_file_id(id_image_link)
            download_url = construct_direct_download_url(file_id)
            
            # Clear the download folder
            download_folder = 'download'
            if not os.path.exists(download_folder):
                os.makedirs(download_folder)
            clear_download_folder(download_folder)
            
            # Download the image
            output_path = os.path.join(download_folder, 'download_image.jpg')
            download_image(download_url, output_path)
        else:
            print("ID image link not found for this user.")
    else:
        print("User not found with the given NIC number.")

final_output("95223342")
# if __name__ == "__main__":
#     main()

Image successfully downloaded: download\download_image.jpg


In [11]:
from test_verification import final_output

final_output("95223342")

Image successfully downloaded: download\download_image.jpg


In [7]:
import cv2
import pytesseract
import re

# Ensure pytesseract can find the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply bilateral filter to reduce noise and keep edges sharp
    filtered = cv2.bilateralFilter(gray, 11, 17, 17)
    
    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    
    return thresh

def extract_name(image):
    # Use Tesseract to do OCR on the image
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(image, config=custom_config)
    
    # Use regex to find the pattern starting with "1,2."
    name_pattern = r"1,2\.\s*([A-Z\s]+)"
    match = re.search(name_pattern, text)
    
    if match:
        return match.group(1).strip()
    return "Name not found"

def extract_id(image):
    # Use Tesseract to do OCR on the image
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(image, config=custom_config)
    
    # Use regex to find the pattern starting with "4d." and ending with a space
    # id_pattern = r"4d\.\s*(\d+)"
    id_pattern = r"4d\.\s*(\d+\s*V?)"
    match = re.search(id_pattern, text)
    
    if match:
        return match.group(1).strip()
    return "ID not found"

def main(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    
    # Extract the name from the preprocessed image
    name = extract_name(preprocessed_image)
    id = extract_id(preprocessed_image)
    
    print(f"Your Name: {name}")
    print(f"Your NIC: {id}")

# Path to the uploaded image
image_path = 'licence1.jpg'
main(image_path)


Your Name: MILLAWA ARACHCHILAGE KANISHKA
CHATHURANGA
Your NIC: 951222372 V


In [2]:
print(pytesseract)

<module 'pytesseract' from 'c:\\Users\\KanishkaChathuranga\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pytesseract\\__init__.py'>


In [1]:
from ice_cream_store_app import main

image_path = 'licence1.jpg'
call_main = main(image_path)
print(call_main)

Your Name: MILLAWA ARACHCHILAGE KANISHKA
CHATHURANGA 
 Your NIC: 951222372 V


In [1]:
import cv2
import chainlit as cl

# Function to capture a selfie from the live camera
def capture_selfie():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the frame
        cv2.imshow('Press Space to Capture', frame)

        # Wait for the space key to capture the image
        if cv2.waitKey(1) & 0xFF == ord(' '):
            captured_image_path = 'selfie.jpg'
            cv2.imwrite(captured_image_path, frame)
            break

    cap.release()
    cv2.destroyAllWindows()
    return captured_image_path

@cl.on_message
def main(message: str):
    # Capture the selfie
    selfie_path = capture_selfie()

    # Check if the selfie was captured successfully
    if selfie_path:
        cl.send_text(f"Selfie captured and saved to {selfie_path}")
        # Send the image to Chainlit
        cl.send_image(selfie_path)
    else:
        cl.send_text("Failed to capture selfie.")

if __name__ == "__main__":
    cl.run()

2024-05-31 09:37:58 - Loaded .env file


KeyError: 'run'

In [13]:
number = int(input())
number%2

1